The Notebook explores and showcases Global Wildfire events

In [1]:
import os
import glob
import pandas as pd
import geopandas as gpd
import dask_geopandas as dgpd
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

In [2]:
cluster = LocalCluster()
client = Client(cluster)
print(f"Dask Dashboard: {client.dashboard_link}")

/home/rufaib/miniconda3/envs/pace_chl/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42181 instead
  warnings.warn(


Dask Dashboard: http://127.0.0.1:42181/status


In [ ]:
yearly_fire = {
    "2015": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2015.shp", 
    "2016": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2016.shp",
    "2017": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2017.shp",
    "2018": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2018.shp",
    "2019": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2019.shp",
    "2020": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2020.shp", 
    "2021": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2021.shp", 
    "2022": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2022.shp", 
    "2023": "/workspace/_output/GLOBFIRE_burned_area_full_dataset_2002_2023/original_globfire_filtered_2023.shp"
}

years = sorted(yearly_fire.keys())

In [37]:
data_paths = yearly_fire
filtered_gdf = dgpd.read_file(data_paths[years[0]], chunksize=2048)


def select_unique_intersections(gdf):
    """Finds intersecting indices to remove"""
    gdf = gdf[gdf.index != gdf.index_right]
    return gdf.index
    
for i in range(1, len(years)):
    current_year = years[i]
    print(f"Processing: {years[i-1]} vs {current_year}")
        
    # Load the next year
    next_year_gdf = dgpd.read_file(data_paths[current_year], chunksize=2048)
        
    print(f"Checking for intersecting geometries: {years[i-1]} vs {current_year}")
    intersections = dgpd.sjoin(filtered_gdf, next_year_gdf, how="inner", predicate="intersects")
        
    print(f"Computing indices of intersecting geometries: {years[i-1]} vs {current_year}")
    intersecting_ids = intersections.map_partitions(select_unique_intersections).compute()
        
    print(f"Removing intersections from {years[i-1]}")
    filtered_gdf = filtered_gdf.map_partitions(lambda df: df[~df.index.isin(intersecting_ids)])
        
    print(f"Filtering non-intersecting geometries from {current_year}")
    non_intersecting_next_year = next_year_gdf.map_partitions(lambda df: df[~df.index.isin(intersecting_ids)])
        
    print(f"Appending non-intersecting geometries from {current_year} to the main dataset")
    filtered_gdf = dgpd.GeoDataFrame(dd.concat([filtered_gdf, non_intersecting_next_year], axis=0))
        
    # print(f"Remaining records after processing {current_year}: {filtered_gdf.compute().shape[0]}")git
    
final_filtered_gdf = filtered_gdf.compute()
final_filtered_gdf.to_file("final_non_intersecting_wildfires_2015_2023.shp")

Processing: 2021 vs 2023
Checking for intersecting geometries: 2021 vs 2023
Computing indices of intersecting geometries: 2021 vs 2023


/home/rufaib/miniconda3/envs/pace_chl/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 139.33 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
2025-02-27 01:40:54,469 - distributed.worker - ERROR - failed during get data with tcp://127.0.0.1:37357 -> tcp://127.0.0.1:37703
Traceback (most recent call last):
  File "/home/rufaib/miniconda3/envs/pace_chl/lib/python3.12/site-packages/tornado/iostream.py", line 861, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
                 ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/rufaib/miniconda3/envs/pace_chl/lib/python3.12/site-packages/tornado/iostream.py", line 1116, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
           ^^^^^^^^^^^^^

Removing intersections from 2021
Filtering non-intersecting geometries from 2023
Appending non-intersecting geometries from 2023 to the main dataset


/home/rufaib/miniconda3/envs/pace_chl/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 9.77 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/rufaib/miniconda3/envs/pace_chl/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field initialdat create as date field, though DateTime requested.
  ogr_write(
/home/rufaib/miniconda3/envs/pace_chl/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field finaldate create as date field, though DateTime requested.
  ogr_write(


In [41]:
unique_fire_df = dgpd.read_file("/workspace/_output/unique_fires/final_non_intersecting_wildfires_2015_2023.shp", chunksize=2048)
unique_fire_df = unique_fire_df.compute()
unique_fire_df.head()

,_uid_,id,initialdat,finaldate,area_ha,geometry
0,19.0,17157261.0,2015-02-27,2015-02-27,42.756104,"POLYGON ((-11.62481 9.70833, -11.61636 9.70833..."
1,152.0,17157623.0,2015-02-25,2015-02-25,42.754171,"POLYGON ((-10.14458 9.10833, -10.13614 9.10833..."
2,174.0,17157680.0,2015-02-27,2015-02-27,21.376840,"POLYGON ((-11.03013 8.95, -11.02591 8.95, -11...."
3,181.0,17157690.0,2015-02-27,2015-02-27,21.376795,"POLYGON ((-10.95755 8.92083, -10.95333 8.92083..."
4,213.0,17157734.0,2015-02-27,2015-03-08,320.648062,"MULTIPOLYGON (((-11.34913 8.7625, -11.34491 8...."


In [42]:
unique_fire_df.sample(20)

,_uid_,id,initialdat,finaldate,area_ha,geometry
317159,486773.0,20541046.0,2017-12-09,2017-12-09,21.374974,"POLYGON ((2.89235 7.64167, 2.89656 7.64167, 2...."
193422,120261.0,18397511.0,2016-01-28,2016-01-28,21.371343,"POLYGON ((28.30401 4.05, 28.30819 4.05, 28.308..."
571418,515044.0,22446624.0,2019-08-02,2019-08-04,406.919986,"MULTIPOLYGON (((46.03365 -23.85833, 46.03821 -..."
2052243,602112.0,26826569.0,2023-09-07,2023-09-07,21.416718,"POLYGON ((-55.66793 -23.83333, -55.66338 -23.8..."
1047706,704272.0,24762664.0,2021-09-17,2021-09-19,42.757849,"POLYGON ((35.74595 -10.21667, 35.75019 -10.216..."
1695798,202166.0,26345420.0,2023-04-29,2023-04-29,21.407488,"POLYGON ((-88.6069 21.23333, -88.60243 21.2333..."
1592466,90579.0,26223066.0,2023-02-13,2023-02-14,192.389381,"POLYGON ((13.67254 8.775, 13.67238 8.77083, 13..."
2199814,777344.0,27011319.0,2023-10-17,2023-10-21,235.087481,"MULTIPOLYGON (((121.78798 -4.55, 121.79216 -4...."
2041237,588932.0,26797320.0,2023-08-10,2023-08-17,149.775360,"POLYGON ((35.1657 -17.75, 35.16652 -17.75417, ..."
113312,700256.0,17959857.0,2015-09-19,2015-09-19,21.465373,"POLYGON ((144.05008 -35.2125, 144.05518 -35.21..."


In [43]:
len(unique_fire_df)

2315073